In [1]:
import Pkg; Pkg.activate(@__DIR__); Pkg.instantiate()
# Pkg.add("Colors")
# Pkg.add("GeometryBasics")
# Pkg.add("RobotDynamics")
# Pkg.add("CoordinateTransformations")
# Pkg.add("Rotations")

  Activating environment at `C:\Users\Matthew\Desktop\Work\CMU\Spring 2023\Optimal Control\Project\Project.toml`


In [2]:
using LinearAlgebra
using ForwardDiff
using BlockDiagonals
using ControlSystems

In [3]:
#Quaternion stuff
function hat(v)
    return [0 -v[3] v[2];
            v[3] 0 -v[1];
            -v[2] v[1] 0]
end
function L(q)
    s = q[1]
    v = q[2:4]
    L = [s    -v';
         v  s*I+hat(v)]
    return L
end
T = Diagonal([1; -ones(3)])
H = [zeros(1,3); I]
function qtoQ(q)
    return H'*T*L(q)*T*L(q)*H
end
function G(q)
    G = L(q)*H
end
function rptoq(ϕ)
    (1/sqrt(1+ϕ'*ϕ))*[1; ϕ]
end
function qtorp(q)
    q[2:4]/q[1]
end

qtorp (generic function with 1 method)

In [49]:
#Quadrotor parameters
m_q = 0.5
m_p = 0.2
ℓ = 0.1750 # Quad radius
J = Diagonal([0.0023, 0.0023, 0.004])
g = 9.81
kt=1.0
km=0.0245
g = [0; 0; 9.81]
L_p = 1 # Pendulum length

h = 0.05 #20 Hz

Nx = 17     # number of states (quaternion)
Nx̃ = 16     # number of states (linearized)
Nu = 4     # number of controls
Tfinal = 5.0 # final time
Nt = Int(Tfinal/h)+1    # number of time steps
thist = Array(range(0,h*(Nt-1), step=h));

In [15]:
function E(q)
    E = BlockDiagonal([1.0*I(3), G(q), 1.0*I(10)])
end

E (generic function with 1 method)

In [50]:
function quad_dynamics(x,u)
    r_q = x[1:3]
    q = x[4:7]/norm(x[4:7]) #normalize q just to be careful
    r_l = x[8:9]
    v_l = x[10:11]
    v_q = x[12:14]
    ω = x[15:17]
    
    ṙ_q = v_q
    q̇ = 0.5*G(q)*ω
    ṙ_l = v_l
        
    B = [I; -r_l' / sqrt(L_p - r_l'*r_l)]
    
    R_ib = qtoQ(q)' # Rotation from body to world frame '
    
    # Calculate Control Thrust and Torques
    F1 = max(0,kt*u[1])
    F2 = max(0,kt*u[2])
    F3 = max(0,kt*u[3])
    F4 = max(0,kt*u[4])
    F_b = [0.; 0.; F1+F2+F3+F4] #total rotor force in body frame
    F = -R_ib * F_b # negative since z axis in body frame is down
    
    M1 = km*u[1]
    M2 = km*u[2]
    M3 = km*u[3]
    M4 = km*u[4]
    M_b = [ℓ*(F3-F1), ℓ*(F2-F4), (M1-M2+M3-M4)] #total rotor torque in body frame
    # paper has two extra terms one for air drag (we ignore) and another for gyroscopic torque
    # Zac seems to ignore gyroscopic but not sure why
            
    
    # TODO: double check Ḃ (I plugged in the scalar form into wolfram alpha and took the derivative)
    Ḃ = [0 0; 0 0; L_p^2 * v_l' / (L_p^2 - r_l' * r_l)^(3/2)]
    ω_skew = hat(ω)

    RHS = [m_p * B' * g; (m_q + m_p) * g; zeros(3)] + [zeros(2); F; M_b]    
    Const = [-m_p * B' * Ḃ * v_l; m_p * Ḃ * v_l; ω_skew * J * ω]

    A = zeros(eltype(x), 8,8)
    A[1:2, :] = [m_p*B'*B m_p*B' zeros(2,3)]
    A[3:5, :] = [m_p*B (m_q+m_p)*I(3) zeros(3,3)]
    A[6:8, :] = [zeros(3,5) J]

    derivs = A \ (RHS - Const)
    
    return [ṙ_q; q̇; ṙ_l; derivs]
end

quad_dynamics (generic function with 1 method)

In [51]:
function quad_dynamics_rk4(x,u)
    #RK4 integration with zero-order hold on u
    f1 = quad_dynamics(x, u)
    f2 = quad_dynamics(x + 0.5*h*f1, u)
    f3 = quad_dynamics(x + 0.5*h*f2, u)
    f4 = quad_dynamics(x + h*f3, u)
    xn = x + (h/6.0)*(f1 + 2*f2 + 2*f3 + f4)
    xn[4:7] .= xn[4:7]/norm(xn[4:7]) #re-normalize quaternion
    return xn
end

quad_dynamics_rk4 (generic function with 1 method)

In [52]:
#Initial Conditions
uhover = ((m_q + m_p)*9.81/4)*ones(4)
r_q0 = [0.0; 0; 1.0]
q0 = [1.0; 0; 0; 0]
r_l0 = [0.0; 0.0]
v_q0 = zeros(3)
v_l0 = zeros(2)
ω0 = zeros(3)
x0 = [r_q0; q0; r_l0; v_q0; v_l0; ω0];

#Dynamics test
x1 = quad_dynamics_rk4(x0,uhover)

17-element Vector{Float64}:
 0.0
 0.0
 1.0
 1.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [53]:
#Linearize dynamics about hover
A = ForwardDiff.jacobian(x->quad_dynamics_rk4(x,uhover),x0)
B = ForwardDiff.jacobian(u->quad_dynamics_rk4(x0,u),uhover);

In [54]:
rank(A)#The naive A matrix will always be rank deficient due to the unit-norm constraint on the quaternion

16

In [55]:
#Reduced system
Ã = Array(E(q0)'*A*E(q0))
B̃ = Array(E(q0)'*B);

In [56]:
#Controlability Matrix
C = B̃
for k = 1:(Nx-1)
    C = [C Ã*C[:,end-(Nu-1):end]]
end

In [57]:
rank(C) #Reduced system is controllable

16

In [58]:
# Cost weights
Q = Array(I(Nx̃));
R = Array(.1*I(Nu));

In [59]:
#LQR Controller
K = dlqr(Ã,B̃,Q,R)

4×16 Matrix{Float64}:
  5.65084e-14   0.107799     -1.2342  …   1.41172e-14   0.681679
  0.107799     -7.82552e-13  -1.2342      0.155001     -0.681679
 -1.86832e-14  -0.107799     -1.2342      3.39659e-15   0.681679
 -0.107799     -5.64065e-13  -1.2342     -0.155001     -0.681679

In [60]:
#Feedback controller
function controller(x)
    r_q = x[1:3]
    q = x[4:7]
    r_l = x[8:9]
    v_l = x[10:11]
    v_q = x[12:14]
    ω = x[15:17]
    
    ϕ = qtorp(L(q0)'*q)
    
    Δx̃ = [r_q - r_q0; ϕ; r_l - r_l0; v_l - v_l0; v_q - v_q0; ω - ω0]
    
    u = uhover - K*Δx̃
end

controller (generic function with 1 method)

In [158]:
#Simulation
uhist = zeros(Nu,Nt)
xhist = zeros(Nx,Nt)
# xhist[:,1] = [r0+randn(3); L(q0)*rptoq([1; 0; 0]); v0; ω0]
xhist[:,1] = x0

# Random starting location + angular offset
xhist[1:3,1] += randn(3)
# xhist[4:7,1] += L(q0)*rptoq([1; 0; 0])

for k = 1:(Nt-1)
    uhist[:,k] = controller(xhist[:,k])
    xhist[:,k+1] = quad_dynamics_rk4(xhist[:,k],uhist[:,k])
end

In [354]:
#Set up visualization
using MeshCat
using StaticArrays
using RobotZoo:Quadrotor

vis = Visualizer()
render(vis)

┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
└ http://127.0.0.1:8706


MeshCat.DisplayedVisualizer(MeshCat.CoreVisualizer(MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()), Set{Any}(), ip"127.0.0.1", 8706))

In [429]:
using CoordinateTransformations, Rotations, Colors
using GeometryBasics
import RobotDynamics as RD

function set_mesh!(vis, model, length)
    
    obj = joinpath(@__DIR__, "quadrotor_scaled.obj")
    
    robot_obj = MeshFileGeometry(obj)
    quad_mat = MeshPhongMaterial(color=colorant"black")
    
    pole_mat = MeshPhongMaterial(color=colorant"blue")
    mass_mat = MeshPhongMaterial(color=colorant"red")
    pole = Cylinder(Point3f0(0,0,0),Point3f0(0,0,-length),0.01f0)
    mass = HyperSphere(Point3f0(0,0,-length), 0.05f0)
    
    setobject!(vis["quad"], robot_obj, quad_mat)
    setobject!(vis["pole"], pole, pole_mat)
    setobject!(vis["mass"], mass, mass_mat)
end

function visualize!(vis, model, x::StaticVector, a)
    quad_x, quad_y, quad_z = x[1], x[2], x[3]
    mass_z = sqrt(L_p^2 - x[8:9]' * x[8:9]) + quad_z
    mass_x, mass_y = x[8] + quad_x, x[9] + quad_y;
#     quad_rot_mat = qtoQ(x[4:7])
    quad_rot_mat = I(3)
    
    up = [0;0;1]
    dir = [quad_x; quad_y; quad_z] - [mass_x; mass_y; mass_z]
    q = [0; cross(up, dir)]
    q[1] = sqrt(norm(up)^2 * norm(dir)^2) + dot(up, dir)
    q = normalize(q)
#     pole_rot_mat = qtoQ(q)
    pole_rot_mat = I(3)
    
    settransform!(vis["quad"], compose(Translation(0,0,a), LinearMap(quad_rot_mat)))
    settransform!(vis["pole"], compose(Translation(quad_x,quad_y,quad_z), LinearMap(pole_rot_mat)))
    settransform!(vis["mass"], Translation(mass_x,mass_y,0))
end

function visualize!(vis, model, tf::Real, X)
    fps = Int(round((length(X)-1)/tf))
    anim = MeshCat.Animation(fps)
    x_dim = size(X[1])[1]
    for (k,x) in enumerate(X)
        atframe(anim, k) do
            x = X[k]
            visualize!(vis, model, SVector{x_dim}(x), k * 0.01) 
        end
    end
    setanimation!(vis, anim)
end

visualize! (generic function with 3 methods)

In [430]:
model = Quadrotor()
set_mesh!(vis, model, L_p)

MeshCat Visualizer with path /meshcat/mass at http://127.0.0.1:8706

In [431]:
x_dim = size(xhist)[1]
X1 = [SVector{x_dim}(x) for x in eachcol(xhist)];
visualize!(vis, model, thist[end], X1)